In [46]:
import os
import numpy as np
import pandas as pd
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss, SMAPE, MAE, RMSE, MAPE
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, Callback
from torch.utils.data import DataLoader
import time

In [47]:

# Set CUDA device if needed
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [48]:
df = pd.read_csv("data/timexer_lambda.csv", parse_dates=["date"])
df = df.sort_values("date")
df["time_idx"] = (df["date"] - df["date"].min()).dt.total_seconds().astype(int)
df["series_id"] = 0

In [49]:

# Split: 70% train, 10% val, 20% test
num_train = int(len(df) * 0.7)
num_val = int(len(df) * 0.1)
num_test = len(df) - num_train - num_val
test_df = df.iloc[num_train + num_val:]



In [50]:
max_encoder_length = 50
batch_size = 64
group_ids = ["series_id"]
target = "duration_sum"
epochs = 20

In [51]:
time_varying_known_reals = [
    "bytes_op0", "bytes_op1", "bytes_sum", "io_count",
    "read_ops_count", "write_ops_count",
    "bytes_sum_ema_short", "bytes_sum_ema_long",
    "bytes_sum_macd", "bytes_sum_macd_signal",
]
time_varying_unknown_reals = ["duration_sum"]
static_categoricals = []
static_reals = []

covariate_scalers = {
    var: GroupNormalizer(groups=group_ids)
    for var in time_varying_known_reals
}

# Define horizons to evaluate
horizons = [1, 5, 10, 15, 20]

In [52]:
model_kwargs = dict(
    learning_rate=1e-4,  # lower LR for fine-tuning
    hidden_size=128,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=128,
    loss=QuantileLoss(quantiles=[0.1, 0.5, 0.9]),
    output_size=3,
    logging_metrics=[SMAPE(), MAE(), RMSE(), MAPE()],
    log_interval=10,
    reduce_on_plateau_patience=4,
)

In [53]:
trainer_kwargs = dict(
    accelerator="gpu",
    devices=1,
    gradient_clip_val=0.1,
    max_epochs=epochs,
)

In [54]:
class PrintLossCallback(Callback):
    def __init__(self, horizon):
        self.horizon = horizon

    def on_train_epoch_end(self, trainer, pl_module):
        epoch = trainer.current_epoch
        metrics = trainer.callback_metrics
        print(f"[H={self.horizon}] Epoch={epoch:02d}, train_loss={metrics.get('train_loss'):.4f}, val_loss={metrics.get('val_loss'):.4f}")

In [55]:
class TimeAndLossCallback(Callback):
    def __init__(self, horizon):
        self.horizon = horizon
        self.epoch_times = []

    def on_train_epoch_start(self, trainer, pl_module):
        self.start_time = time.time()

    def on_train_epoch_end(self, trainer, pl_module):
        end_time = time.time()
        elapsed = end_time - self.start_time
        self.epoch_times.append(elapsed)

        epoch = trainer.current_epoch
        metrics = trainer.callback_metrics
        print(
            f"[H={self.horizon}] Epoch={epoch:02d}, "
            f"train_loss={metrics.get('train_loss'):.4f}, "
            f"val_loss={metrics.get('val_loss'):.4f}, "
            f"time={elapsed:.2f}s"
        )

    def on_train_end(self, trainer, pl_module):
        avg_time = np.mean(self.epoch_times)
        print(f"🕒 [H={self.horizon}] Average epoch time: {avg_time:.2f} seconds over {len(self.epoch_times)} epochs")


In [56]:
horizons = [1,5, 10, 15, 20]
for horizon in horizons:
    print(f"\n🔁 Fine-tuning model for horizon = {horizon}")
    ckpt_path = f"checkpoints_no_static/horizon_{horizon}"
    best_ckpt_file = [f for f in os.listdir(ckpt_path) if f.endswith(".ckpt")]
    assert len(best_ckpt_file) == 1, f"Checkpoint missing in {ckpt_path}"
    best_ckpt = os.path.join(ckpt_path, best_ckpt_file[0])

    training = TimeSeriesDataSet(
        train_df,
        time_idx="time_idx",
        target=target,
        group_ids=group_ids,
        max_encoder_length=max_encoder_length,
        max_prediction_length=horizon,
        static_categoricals=static_categoricals,
        static_reals=static_reals,
        time_varying_known_reals=time_varying_known_reals,
        time_varying_unknown_reals=time_varying_unknown_reals,
        target_normalizer=GroupNormalizer(groups=group_ids),
        scalers=covariate_scalers,
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )
    validation = TimeSeriesDataSet.from_dataset(training, val_df, stop_randomization=True)

    train_loader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_loader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # model = TemporalFusionTransformer.load_from_checkpoint(best_ckpt)
    

    ckpt_dir = f"ckpts_lambda/horizon_{horizon}_lambda"
    os.makedirs(ckpt_dir, exist_ok=True)
    early_stop = EarlyStopping(monitor="val_loss", patience=5, mode="min")
    checkpoint = ModelCheckpoint(
        dirpath=ckpt_dir,
        filename="finetuned-{epoch:02d}-{val_loss:.4f}",
        save_top_k=1,
        monitor="val_loss",
        mode="min",
    )
    time_cb = TimeAndLossCallback(horizon)
    model = TemporalFusionTransformer.from_dataset(training, **model_kwargs )
    trainer = Trainer(callbacks=[early_stop, checkpoint, time_cb], **trainer_kwargs)
    trainer.fit(model, train_loader, val_loader)

    print(f"✅ Fine-tuning complete for horizon {horizon}. Saved to {ckpt_dir}")


🔁 Fine-tuning model for horizon = 1


/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 0      | train
3  | prescalers                         | ModuleDict               

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.
/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

[H=20] Epoch=04, train_loss=0.0002, val_loss=0.0027, time=278.73s


Validation: |          | 0/? [00:00<?, ?it/s]

[H=20] Epoch=05, train_loss=0.0002, val_loss=0.0028, time=276.95s


Validation: |          | 0/? [00:00<?, ?it/s]

[H=20] Epoch=06, train_loss=0.0002, val_loss=0.0027, time=291.25s
🕒 [H=20] Average epoch time: 280.74 seconds over 7 epochs
✅ Fine-tuning complete for horizon 20. Saved to ckpts_lambda/horizon_20_lambda


In [57]:
for horizon in horizons:
    print(f"\n📊 Evaluating fine-tuned model for horizon = {horizon}")

    # Load original training dataset just for consistent normalization
    train_df = df.iloc[:num_train]
    training = TimeSeriesDataSet(
        train_df,
        time_idx="time_idx",
        target=target,
        group_ids=group_ids,
        max_encoder_length=max_encoder_length,
        max_prediction_length=horizon,
        static_categoricals=static_categoricals,
        static_reals=static_reals,
        time_varying_known_reals=time_varying_known_reals,
        time_varying_unknown_reals=time_varying_unknown_reals,
        target_normalizer=GroupNormalizer(groups=group_ids),
        scalers=covariate_scalers,
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )

    # Build test set using from_dataset
    test = TimeSeriesDataSet.from_dataset(training, test_df, stop_randomization=True)
    test_loader = test.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load fine-tuned model
    ckpt_dir = f"ckpts_lambda/horizon_{horizon}_lambda"
    ckpt_files = [f for f in os.listdir(ckpt_dir) if f.endswith(".ckpt")]
    assert len(ckpt_files) == 1, f"Expected one .ckpt file in {ckpt_dir}"
    best_ckpt = os.path.join(ckpt_dir, ckpt_files[0])
    model = TemporalFusionTransformer.load_from_checkpoint(best_ckpt)

    # Predict
    preds = model.predict(test_loader, return_x=False).detach().cpu().numpy().flatten()

    # True labels
    actuals = np.concatenate([y.detach().cpu().numpy()
                              for _, (y, _) in iter(test_loader)], axis=0).flatten()

    # Metrics
    mse = mean_squared_error(actuals, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals, preds)
    mape = mean_absolute_percentage_error(actuals, preds)
    smape = 100 * np.mean(2 * np.abs(actuals - preds) / (np.abs(actuals) + np.abs(preds) + 1e-8))

    print(f"✅ Test Metrics (horizon={horizon}):")
    print(f"   MSE   = {mse:.6f}")
    print(f"   RMSE  = {rmse:.6f}")
    print(f"   MAE   = {mae:.6f}")
    print(f"   MAPE  = {mape:.6%}")
    print(f"   SMAPE = {smape:.6f}%")


📊 Evaluating fine-tuned model for horizon = 1


/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DE

✅ Test Metrics (horizon=1):
   MSE   = 0.003439
   RMSE  = 0.058643
   MAE   = 0.004061
   MAPE  = 37.187183%
   SMAPE = 14.897680%

📊 Evaluating fine-tuned model for horizon = 5


/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DE

✅ Test Metrics (horizon=5):
   MSE   = 0.003477
   RMSE  = 0.058965
   MAE   = 0.004154
   MAPE  = 40.143099%
   SMAPE = 15.814334%

📊 Evaluating fine-tuned model for horizon = 10


...

✅ Test Metrics (horizon=10):
   MSE   = 0.003492
   RMSE  = 0.059091
   MAE   = 0.004196
   MAPE  = 33.164391%
   SMAPE = 18.245224%

📊 Evaluating fine-tuned model for horizon = 15


/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DE

✅ Test Metrics (horizon=15):
   MSE   = 0.003553
   RMSE  = 0.059607
   MAE   = 0.004204
   MAPE  = 32.559642%
   SMAPE = 16.431154%

📊 Evaluating fine-tuned model for horizon = 20


/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/unt.ad.unt.edu/srr0248/.conda/envs/pytorch-forecasting-dev/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DE

✅ Test Metrics (horizon=20):
   MSE   = 0.003497
   RMSE  = 0.059138
   MAE   = 0.004204
   MAPE  = 30.333918%
   SMAPE = 18.002275%
